I decided to predict whether a person will get a loan or not based on the information available. First, we have to determine which variables we want to include. Since we need the same set of variables from the accepted and rejected set, we cannot use any variables that are not in both. The rejected set contains only nine variables. From these nine variables, I could not find their equivalent in the accepted dataset for two of them: date and risk score. While they would have been interesting to examine, I could not use them. Further, the policy code is always 0 in the rejected dataset, so I dropped this column as well. From the over 150 variables in the accept set, I dropped all but the six useful ones in the reject dataset. This left me with the variables: amount asked for, loan title, zip code, state, employment length, and dept to income ratio. I added one column that indicates whether the loan was accepted or not.     

In the next step, I tried to clean the data. I examined which variables had missing values. I found that especially the amount applied had missing values. Since the dataset is very large, and I suspect that the amount applied could have an impact on the outcome, I decided to drop all rows with missing values. This will make it easier to fit and evaluate a model. Next, I deleted the additional percent sign that the rejected dataset had for dti values. After this, both datasets had comparable values, and I merged them into one big dataset.        
I normalized the amount applied for and the dti, to avoid that the big values of the amount applied for skew the model. I one hot encoded the categorical variables. I decided that state, zip code, and employment length are categorical. I included employment length because people who work less than a year or more than 10 years at their current job do not have exact values. Therefore, numerical values would not have been exact and could have lead to mistakes. The categorical variable in one hot encoding can assign a different value for long-term workers than for workers that work for 9 years at their job. Lastly, I used a bag of words to evaluate the title of the loan. I decided to include the 100 most used words. I did that to limit the number of variables and to have enough data per word to find a reasonable coefficient.         
I split my data into training and test set. I decided to fit a logistic regression model to predict the chance of getting a loan. This provides more useful information than other categorical models. I used cross-validation to evaluate if balanced class weights or even class weights are better. The difference between the two options was minimal, but even weights performed slightly better. I used cross-validation instead of the test set to avoid overfitting. Lastly, I calculated the accuracy, recall, and precision of my model on the test set. All were over 90%, so the model seems to be fairly accurate.      

Code: 

In [1]:
#import all packages needed

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [89]:


#load both datasets
data_acc = pd.read_csv('accepted_2007_to_2018Q4.csv.gz', low_memory=False, nrows = 10000)
data_rej = pd.read_csv('rejected_2007_to_2018Q4.csv.gz', low_memory=False, nrows = 10000)

In [90]:
#examine data
data_acc.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
#examine data
data_rej.head()

,Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,10%,481xx,NM,4 years,0.0
1,1000.0,2007-05-26,Consolidating Debt,703.0,10%,010xx,MA,< 1 year,0.0
2,11000.0,2007-05-27,Want to consolidate my debt,715.0,10%,212xx,MD,1 year,0.0
3,6000.0,2007-05-27,waksman,698.0,38.64%,017xx,MA,< 1 year,0.0
4,1500.0,2007-05-27,mdrigo,509.0,9.43%,209xx,MD,< 1 year,0.0


In [92]:
#drop all columns that are not needed from accepted dataset and add acepted as column

data_acc.drop(data_acc.columns.difference(['funded_amnt','title', 'dti', 'zip_code', 'addr_state', 'emp_length']), 1, inplace=True)
data_acc["accepted"] = [1 for i in range(len((data_acc['funded_amnt'])))]

In [93]:
#examine changes
data_acc.head()

,funded_amnt,emp_length,title,zip_code,addr_state,dti,accepted
0,3600.0,10+ years,Debt consolidation,190xx,PA,5.91,1
1,24700.0,10+ years,Business,577xx,SD,16.06,1
2,20000.0,10+ years,NaN,605xx,IL,10.78,1
3,35000.0,10+ years,Debt consolidation,076xx,NJ,17.06,1
4,10400.0,3 years,Major purchase,174xx,PA,25.37,1


In [94]:
#drop all columns that are not needed from rejected dataset and add acepted as column
data_rej.drop(data_rej.columns.difference(['Amount Requested','Loan Title', 'Debt-To-Income Ratio', 'Zip Code', 'State', 'Employment Length']), 1, inplace=True)
data_rej["accepted"] = [0 for i in range(len((data_rej['Loan Title'])))]
#rename rejected dataset to match accepted dataset 
data_rej = data_rej.rename(columns={'Amount Requested': 'funded_amnt','Loan Title': 'title', 'Debt-To-Income Ratio': 'dti', 'Zip Code' : 'zip_code', 'State' : 'addr_state', 'Employment Length' : 'emp_length'})


In [95]:
#examine changes
data_rej.head()

,funded_amnt,title,dti,zip_code,addr_state,emp_length,accepted
0,1000.0,Wedding Covered but No Honeymoon,10%,481xx,NM,4 years,0
1,1000.0,Consolidating Debt,10%,010xx,MA,< 1 year,0
2,11000.0,Want to consolidate my debt,10%,212xx,MD,1 year,0
3,6000.0,waksman,38.64%,017xx,MA,< 1 year,0
4,1500.0,mdrigo,9.43%,209xx,MD,< 1 year,0


In [96]:
#examine missing values
print(data_acc.isna().sum())
print(data_rej.isna().sum())

funded_amnt      0
emp_length     549
title          132
zip_code         0
addr_state       0
dti              0
accepted         0
dtype: int64
funded_amnt    0
title          0
dti            0
zip_code       0
addr_state     0
emp_length     0
accepted       0
dtype: int64


In [97]:
#drop missing values
data_acc.dropna( inplace=True)
data_rej.dropna( inplace=True)

In [98]:
#examine missing values again
print(data_acc.isna().sum())
print(data_rej.isna().sum())

funded_amnt    0
emp_length     0
title          0
zip_code       0
addr_state     0
dti            0
accepted       0
dtype: int64
funded_amnt    0
title          0
dti            0
zip_code       0
addr_state     0
emp_length     0
accepted       0
dtype: int64


In [99]:
#drop percentage sign from rejected dataset dti column
def clean(x):
    x = x.replace("%", "")
    return float(x)
data_rej['dti'] = data_rej['dti'].apply(clean)

print(data_rej['dti'])

0       10.00
1       10.00
2       10.00
3       38.64
4        9.43
        ...  
9995    24.69
9996     0.00
9997     4.56
9998    16.95
9999     8.60
Name: dti, Length: 10000, dtype: float64


In [100]:
#merge datasets to one
data_all = pd.concat([data_acc, data_rej], ignore_index=True)
print(data_all.head())
data_all.shape

   funded_amnt emp_length               title zip_code addr_state    dti  \
0       3600.0  10+ years  Debt consolidation    190xx         PA   5.91   
1      24700.0  10+ years            Business    577xx         SD  16.06   
2      35000.0  10+ years  Debt consolidation    076xx         NJ  17.06   
3      10400.0    3 years      Major purchase    174xx         PA  25.37   
4      11950.0    4 years  Debt consolidation    300xx         GA  10.20   

   accepted  
0         1  
1         1  
2         1  
3         1  
4         1  


(19326, 7)

In [101]:
#normalize funded amount and dti
fa =  np.array(data_all["funded_amnt"])
fa = fa.reshape(-1, 1) 
min_max_scaler = preprocessing.MinMaxScaler()
fa_scaled = min_max_scaler.fit_transform(fa)

dti =  np.array(data_all["dti"])
dti = fa.reshape(-1, 1) 
min_max_scaler = preprocessing.MinMaxScaler()
dti_scaled = min_max_scaler.fit_transform(dti)

data_all['dti'] = dti_scaled
data_all['funded_amnt'] = fa_scaled
print(data_all.head())

   funded_amnt emp_length               title zip_code addr_state       dti  \
0     0.089855  10+ years  Debt consolidation    190xx         PA  0.089855   
1     0.701449  10+ years            Business    577xx         SD  0.701449   
2     1.000000  10+ years  Debt consolidation    076xx         NJ  1.000000   
3     0.286957    3 years      Major purchase    174xx         PA  0.286957   
4     0.331884    4 years  Debt consolidation    300xx         GA  0.331884   

   accepted  
0         1  
1         1  
2         1  
3         1  
4         1  


In [102]:
#use hot one encoding for categorical data and add new columns to dataframe while dropping old ones
data_zip = pd.get_dummies(data_all['zip_code'])
data_state = pd.get_dummies(data_all['addr_state'])
data_emp = pd.get_dummies(data_all['emp_length'])
print(data_zip.shape)
print(data_state.shape)
print(data_emp.shape)
data_all = data_all.merge(data_zip, left_index=True, right_index=True)
data_all = data_all.merge(data_state, left_index=True, right_index=True)
data_all = data_all.merge(data_emp, left_index=True, right_index=True)
data_all.drop('zip_code', axis=1, inplace=True)
data_all.drop('addr_state', axis=1, inplace=True)
data_all.drop('emp_length', axis=1, inplace=True)
print(data_all.shape)

(19326, 857)
(19326, 51)
(19326, 11)
(19326, 923)


In [103]:
print(data_all.head())

   funded_amnt               title       dti  accepted  008xx  010xx  011xx  \
0     0.089855  Debt consolidation  0.089855         1      0      0      0   
1     0.701449            Business  0.701449         1      0      0      0   
2     1.000000  Debt consolidation  1.000000         1      0      0      0   
3     0.286957      Major purchase  0.286957         1      0      0      0   
4     0.331884  Debt consolidation  0.331884         1      0      0      0   

   012xx  013xx  014xx  ...  10+ years  2 years  3 years  4 years  5 years  \
0      0      0      0  ...          1        0        0        0        0   
1      0      0      0  ...          1        0        0        0        0   
2      0      0      0  ...          1        0        0        0        0   
3      0      0      0  ...          0        0        1        0        0   
4      0      0      0  ...          0        0        0        1        0   

   6 years  7 years  8 years  9 years  < 1 year  
0     

In [104]:
#add word bag for the 100 most frequently used words to dataset and drop ald column

words = [i for i in data_all['title']]

vectorizer = CountVectorizer(stop_words = 'english')
X = vectorizer.fit_transform(words)
#X.get_feature_names_out()

res = X.sum(axis=0)
#print(b.shape)
words_freq = [(word, res[0, idx], idx) for word, idx in vectorizer.vocabulary_.items()]
words_freq_s = sorted(words_freq, key = lambda x: x[1], reverse=True)
X_new = X.toarray().transpose()
count = 0
for i in words_freq_s:
    if count < 100:
        print(i)
        data_all[i[0]] = X_new[i[2]]
    count +=1
data_all.drop('title', axis=1, inplace=True)

('debt', 5589, 2110)
('consolidation', 5483, 1738)
('credit', 2461, 1848)
('card', 2363, 1293)
('refinancing', 2280, 6680)
('debt_consolidation', 1229, 2111)
('home', 605, 3398)
('improvement', 518, 3541)
('loan', 242, 4807)
('purchase', 224, 6486)
('major', 207, 4988)
('major_purchase', 199, 4989)
('business', 156, 1204)
('car', 136, 1289)
('pay', 127, 6164)
('expenses', 124, 2709)
('medical', 119, 5202)
('financing', 101, 2813)
('help', 88, 3323)
('consolidate', 85, 1734)
('home_improvement', 78, 3399)
('need', 77, 5791)
('cards', 71, 1298)
('vacation', 66, 8501)
('moving', 63, 5565)
('personal', 60, 6231)
('high', 54, 3358)
('relocation', 53, 6701)
('money', 52, 5514)
('buying', 50, 1222)
('payment', 43, 6172)
('like', 39, 4722)
('new', 38, 5828)
('paying', 38, 6169)
('bills', 37, 918)
('rate', 32, 6579)
('time', 30, 8125)
('payments', 28, 6173)
('work', 27, 8789)
('000', 27, 1)
('looking', 24, 4831)
('want', 23, 8638)
('school', 23, 7138)
('college', 20, 1674)
('monthly', 20, 5529)

In [105]:
#examine changes
print(data_all.shape)
print(data_all.head())

(19326, 1022)
   funded_amnt       dti  accepted  008xx  010xx  011xx  012xx  013xx  014xx  \
0     0.089855  0.089855         1      0      0      0      0      0      0   
1     0.701449  0.701449         1      0      0      0      0      0      0   
2     1.000000  1.000000         1      0      0      0      0      0      0   
3     0.286957  0.286957         1      0      0      0      0      0      0   
4     0.331884  0.331884         1      0      0      0      0      0      0   

   015xx  ...  track  paid  needed  valley  son  way  investment  repair  old  \
0      0  ...      0     0       0       0    0    0           0       0    0   
1      0  ...      0     0       0       0    0    0           0       0    0   
2      0  ...      0     0       0       0    0    0           0       0    0   
3      0  ...      0     0       0       0    0    0           0       0    0   
4      0  ...      0     0       0       0    0    0           0       0    0   

   fees  
0     0 

In [106]:
#define X and y and split data into training and test data
y = data_all['accepted']
X = data_all.loc[:, data_all.columns != 'accepted']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [107]:
#define logistic regression for no weight and balanced weights 
clf1 = LogisticRegression(random_state=0).fit(X_train, y_train)
clf2 = LogisticRegression(class_weight = 'balanced', random_state=0).fit(X_train, y_train)

#use cross validation to examine different methods
y_pred_1 = cross_val_score(clf1 , X_train, y_train, cv=3)
print(np.mean(y_pred_1))

y_pred_2 = cross_val_score(clf2 , X_train, y_train, cv=3)
print(np.mean(y_pred_2))



0.9672967407650096
0.9671587573905284


In [108]:
#evaluate on test set

y_pred_test = clf1.predict(X_test)

print("accuracy: ", accuracy_score(y_pred_test, y_test))
print("precision score: ", precision_score(y_pred_test, y_test))
print("recall score: ", recall_score(y_pred_test, y_test))

accuracy:  0.9646109271523179
precision score:  0.943404255319149
recall score:  0.9831485587583149
